In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os, librosa, csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten
from tqdm import tqdm

Using TensorFlow backend.


In [24]:
physical_gpus = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_virtual_device_configuration(
    physical_gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)] 
)
logical_gpus = tf.config.list_logical_devices("GPU")

2022-07-16 19:13:03.881031: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-16 19:13:03.881130: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-07-16 19:13:03.881173: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-16 19:13:03.881186: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-16 19:13:03.881198: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-07-16 19:13:03.881210: I tensorflow/stream_executor/

In [2]:
#x_path = "dataset/data_sr/audio_sample.csv"
x_path = "dataset/data_sr/audio_sample.csv"
y_path = "dataset/data_sr/label_sample.csv"

In [3]:
x = pd.read_csv(x_path)
y = pd.read_csv(y_path)

In [4]:
#x["特徵"] = pd.to_numeric(x["特徵"], errors='coerce')
x.head()

,檔名,特徵
0,1631855145,[[ 0.12715088 0.12715088 0.12715088 ... 0.1...
1,1619349620,[[ 0.06157185 0.06157185 0.06157185 ... 0.0...
2,1618415564,[[ 0.01064791 0.01064791 0.01064791 ... 0.0...
3,1627011377,[[ 0.15257801 0.15257801 0.15257801 ... 0.1...
4,1633141439,[[ 0.04986587 0.04986587 0.04986587 ... 0.0...


In [5]:
print(x.dtypes)

檔名     int64
特徵    object
dtype: object


In [6]:

#x["特徵"] = pd.to_numeric(x["特徵"])

In [9]:
y.head()

,檔名,請開,請關,請保持,其他
0,1633056677,0,0,0,1
1,1648290403,0,0,0,1
2,1651108191,0,0,0,1
3,1632317974,0,0,0,1
4,1637285404,0,0,0,1


In [10]:
df = pd.merge(x,y, how="left", left_on="檔名", right_on="檔名")

In [11]:
df.head()

,檔名,特徵,請開,請關,請保持,其他
0,1631855145,[[ 0.12715088 0.12715088 0.12715088 ... 0.1...,0.0,0.0,0.0,1.0
1,1619349620,[[ 0.06157185 0.06157185 0.06157185 ... 0.0...,1.0,0.0,0.0,0.0
2,1618415564,[[ 0.01064791 0.01064791 0.01064791 ... 0.0...,0.0,0.0,0.0,1.0
3,1627011377,[[ 0.15257801 0.15257801 0.15257801 ... 0.1...,0.0,0.0,0.0,1.0
4,1633141439,[[ 0.04986587 0.04986587 0.04986587 ... 0.0...,0.0,0.0,0.0,1.0


In [12]:
df.shape

(91652, 6)

In [13]:
df = df.dropna()

In [14]:
df.shape

(91110, 6)

In [37]:
# X values -> don't read string, read float
X = df[df.columns[1]].values
y = df[df.columns[2:6]].values

In [16]:
#a = X.tobytes()
#X = np.frombuffer(a, dtype=float)

In [38]:
print(X)

['[[ 0.12715088  0.12715088  0.12715088 ...  0.10713703  0.00662959\n   0.12239534]\n [ 0.12715088  0.12715088  0.12715088 ...  0.1450959  -0.11178505\n  -0.04536566]\n [ 0.12715088  0.12715088  0.12715088 ...  0.15172765  0.05275317\n   0.15467575]\n ...\n [ 1.6946012  -0.009485   -0.5138868  ...  0.12715088  0.12715088\n   0.12715088]\n [ 1.6336005   0.1152609  -0.7340485  ...  0.12715088  0.12715088\n   0.12715088]\n [ 1.6119637  -0.05145656 -0.8914775  ...  0.12715088  0.12715088\n   0.12715088]]'
 '[[ 0.06157185  0.06157185  0.06157185 ...  0.05296574  0.06363794\n  -0.01879503]\n [ 0.06157185  0.06157185  0.06157185 ...  0.07545546  0.14161375\n  -0.04537746]\n [ 0.06157185  0.06157185  0.06157185 ...  0.08640184  0.07667358\n  -0.09694857]\n ...\n [ 0.          0.          0.         ...  0.          0.\n   0.        ]\n [ 0.          0.          0.         ...  0.          0.\n   0.        ]\n [ 0.          0.          0.         ...  0.          0.\n   0.        ]]'
 '[[ 0.010

In [39]:
X.shape, y.shape

((91110,), (91110, 4))

In [40]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

In [41]:
X_train = np.reshape(X_train, (X_train.shape[0], 1))
#X_train.reshape((X_train[0]), 1)
#X_train.reshape((-1, 1, 108, 494))
X_test = np.reshape(X_test, (X_test.shape[0], 1))
X_valid = np.reshape(X_valid, (X_valid.shape[0], 1))

In [42]:
print(X.shape)

(91110,)


In [43]:
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(X_train.shape)))
model.add(Dense(4, activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 54666, 16)         32        
_________________________________________________________________
dense_9 (Dense)              (None, 54666, 4)          68        
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [45]:
model.fit(X_train, y_train, batch_size=16, epochs=20)

ValueError: Error when checking input: expected dense_8_input to have 3 dimensions, but got array with shape (54666, 1)

In [ ]:
model.evaluate(X_train, y_train)

In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, 
               input_shape=(X_train.shape[0], 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(4))
model.summary()

In [ ]:
EMBEDDING_SIZE = 128
HIDDEN_LAYER_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 10

model.add(Embedding(vocab_size, EMBEDDING_SIZE,input_length=MAX_SENTENCE_LENGTH))

In [26]:
X = list(X)
a = []
count = 0
for x in X:
    x = x.strip("[]")
    x = x.strip()
    x = x.split("\n")
    b = []
    for i in x:
        i = i.strip()
        i = i.split(' ')
        b.append(i)
    count += 1
    a.append(b)

X = np.array(a, dtype=object)